In [14]:
import textwrap
from abc import ABC, abstractclassmethod, abstractproperty
from datetime import datetime


class Client:
    def __init__(self, address):
        self.address = address
        self.accounts = []

    def execute_transaction(self, account, transaction):
        transaction.record(account)

    def create_account(self, account):
        self.accounts.append(account)


class NaturalPerson(Client):
    def __init__(self, name, birth_date, document, address):
        super().__init__(address)
        self.name = name
        self.birth_date = birth_date
        self.document = document

class account:
    def __init__(self, account_num, client):
        self._balance = 0
        self._account_num = account_num
        self._agency = "0001"
        self._client = client
        self._statement = Statement()

    @classmethod
    def new_account(cls, client, account_num):
        return cls(account_num, client)

    @property
    def balance(self):
        return self._balance

    @property
    def account_num(self):
        return self._account_num

    @property
    def agency(self):
        return self._agency

    @property
    def client(self):
        return self._client

    @property
    def statement(self):
        return self._statement

    def withdraw(self, amount):
        balance = self._balance
        
        if amount > balance:
            print("\n@@@ Transaction has been failed! You have no enough balance. @@@")

        elif amount > 0:
            self._balance -= amount
            print(f"\n=== withdraw confirmed with success: $ {amount:.2f}===")
            print(f"Balance ==> $ {self._balance:.2f}")
            return True

        else:
            print("\n@@@ Withdraw has been failed! @@@")

        return False

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            print(f"\n=== Deposit confirmed with success: $ {amount:.2f}===")
            print(f"Balance ==> $ {self._balance:.2f}")
            return True

        else:
            print("\n@@@ Deposit has been failed! @@@")

        return False


class CheckAccount(account):
    def __init__(self, account_num, client, limit=500, withdraw_max=3):
        super().__init__(account_num, client)
        self._limit = limit
        self._withdraw_max = withdraw_max

    def withdraw(self, amount):
        account_withdraw_num = len(
            [transaction for transaction in self.statement.transactions if transaction["type"] == Withdraw.__name__]
        )
        
        if account_withdraw_num >= self._withdraw_max:
          print("\n@@@ Transaction has been failed! Maximum number of withdrawals exceeded. @@@") 
 
        elif amount > self._limit:
          print("\n@@@ Transaction has been failed! Withdrawal exceeds the amount limit. @@@")

        else:
            return super().withdraw(amount)

        return False

    def __str__(self):
        return f"""\
            Agency:\t{self.agency}
            Account:\t\t{self.account_num}
            Client:\t{self.client.name}
        """


class Statement:
    def __init__(self):
        self._transactions = []

    @property
    def transactions(self):
        return self._transactions

    def create_transaction(self, transaction):
        self._transactions.append(
            {
                "type": transaction.__class__.__name__,
                "amount": transaction.amount,
                "trans_date": datetime.now().strftime("%d-%m-%Y %H:%M:%S"),
            }
        )


class Transaction(ABC):
    @property
    @abstractproperty
    def amount(self):
        pass

    @abstractclassmethod
    def record(self, account):
        pass


class Withdraw(Transaction):
    def __init__(self, amount):
        self._amount = amount

    @property
    def amount(self):
        return self._amount

    def record(self, account):
        transaction_OK = account.withdraw(self.amount)

        if transaction_OK:
            account.statement.create_transaction(self)


class Deposit(Transaction):
    def __init__(self, amount):
        self._amount = amount

    @property
    def amount(self):
        return self._amount

    def record(self, account):
        transaction_OK = account.deposit(self.amount)

        if transaction_OK:
            account.statement.create_transaction(self)


def menu():
    menu = """\n
    ================ MENU ================
    [1]\tDeposit
    [2]\tWithdraw
    [3]\tStatement
    [4]\tNew client
    [5]\tNew account
    [6]\tList all accounts\n
    
    [7]\tExit
    ======================================
    => """
    return input(textwrap.dedent(menu))



def filter_client(document, clients):
    clients_filter = [client for client in clients if client.document == document]
    return clients_filter[0] if clients_filter else None

def find_account_client(client):
    if not client.accounts:
        print("\n@@@ Client does not have account! @@@")
        return

    # FIXME: does not allow the client chose a account
    return client.accounts[0]


def deposit(clients):
    document = input("Enter the client document (only numbers): ")
    client = filter_client(document, clients)

    if not client:
        print("\n@@@ Client not found! @@@")
        return

    account = find_account_client(client)
    if not account:
        return

    amount = float(input("Enter the deposit amount: "))
    transaction = Deposit(amount)

    client.execute_transaction(account, transaction)


def withdraw(clients):
    document = input("Enter the client document (only numbers): ")
    client = filter_client(document, clients)

    if not client:
        print("\n@@@ Client not found! @@@")
        return

    account = find_account_client(client)
    if not account:
        return

    amount = float(input("Enter the withdraw amount: "))
    transaction = Withdraw(amount)

    client.execute_transaction(account, transaction)


def show_statement(clients):
    document = input("Enter the client document (only numbers): ")
    client = filter_client(document, clients)

    if not client:
        print("\n@@@ Client not found! @@@")
        return

    account = find_account_client(client)
    if not account:
        return

    print("\n================ STATEMENT ================")
    transactions = account.statement.transactions

    statement = ""
    if not transactions:
        statement = "There is no transactions."
    else:
        for transaction in transactions:
            statement += f"\n{transaction['type']}:\tR$ {transaction['amount']:.2f}"

    print(statement)
    print(f"\nbalance:\t $ {account.balance:.2f}")
    print("==========================================")


def create_client(clients):
    document = input("Enter the client document (only numbers): ")
    client = filter_client(document, clients)

    if client:
        print("\n@@@ Client already exists! @@@")
        return

    name = input("Enter the full name: ")
    birth_date = input("Enter the birth date (dd-mm-aaaa): ")
    address = input("Enter the address (street, number - City/Two-letter State Abbr): ")

    client = NaturalPerson(name=name, birth_date=birth_date, document=document, address=address)

    clients.append(client)

    print("=== Client successfully registered! ===")


def create_account(account_num_account, clients, accounts):
    document = input("Enter the client document (only numbers): ")
    client = filter_client(document, clients)

    if not client:
        print("\n@@@ Client not found! Account creation aborted. @@@")
        return
    
    account = CheckAccount.new_account(client=client, account_num=account_num_account)
    accounts.append(account)
    client.accounts.append(account)

    print("=== Account successfully created! ===")


def list_accounts(accounts):
    for account in accounts:
        print("=" * 100)
        print(textwrap.dedent(str(account)))


def main():
    clients = []
    accounts = []

    while True:
        option = menu()

        if option == "1":
            deposit(clients)

        elif option == "2":
            withdraw(clients)

        elif option == "3":
            show_statement(clients)

        elif option == "4":
            create_client(clients)

        elif option == "5":
            account_num = len(accounts) + 1
            create_account(account_num, clients, accounts)

        elif option == "6":
            list_accounts(accounts)

        elif option == "7":
            break

        else:
            print("@@@ Invalid option! Please, select a valid option! @@@")

main()



================ MENU ================
[1]	Deposit
[2]	Withdraw
[3]	Statement
[4]	New client
[5]	New account
[6]	List all accounts


[7]	Exit
=> 4
Enter the client document (only numbers): 44
Enter the full name: 44
Enter the birth date (dd-mm-aaaa): 44
Enter the address (street, number - City/Two-letter State Abbr): 44
=== Client successfully registered! ===


================ MENU ================
[1]	Deposit
[2]	Withdraw
[3]	Statement
[4]	New client
[5]	New account
[6]	List all accounts


[7]	Exit
=> 5
Enter the client document (only numbers): 44
=== Account successfully created! ===


================ MENU ================
[1]	Deposit
[2]	Withdraw
[3]	Statement
[4]	New client
[5]	New account
[6]	List all accounts


[7]	Exit
=> 1
Enter the client document (only numbers): 444

@@@ Client not found! @@@


================ MENU ================
[1]	Deposit
[2]	Withdraw
[3]	Statement
[4]	New client
[5]	New account
[6]	List all accounts


[7]	Exit
=> 5
Enter the client document (only n

KeyboardInterrupt: ignored